## Loading a Fine-Tuned Detectron Model for Auto Labeling
In this notebook, we will load a finely tuned Detectron model and use it to label datasets and upload them back to SegmentsAI.



In [1]:
import numpy as np

# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()


# checking the version and if we have cuda available
from src.segmentation.framework_handlers.detectron2_handler import print_version_info

!nvcc --version
print_version_info()

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Aug_15_21:14:11_PDT_2021
Cuda compilation tools, release 11.4, V11.4.120
Build cuda_11.4.r11.4/compiler.30300941_0
torch:  1.9 ; cuda:  11.1
detectron2: 0.6


In [2]:
# define the time of the training
from datetime import datetime

# Get the current date and time of the model training process
current_time = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

print(f"current time {current_time}")

current time 16-04-2024_16-49-14


Code for loading model checkpoint and config

In [3]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

# load the best model config and weights
checkpoint_path = "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/30-12-2023_12-34-05_all_train_images/model_final.pth"
config_yaml_path = "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/30-12-2023_12-34-05_all_train_images/config.yaml"
cfg = get_cfg()
# load config
cfg.merge_from_file(config_yaml_path)
# define predictor
# load checkpoint
cfg.MODEL.WEIGHTS = checkpoint_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

Loading config /home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/30-12-2023_12-34-05_all_train_images/config.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[04/16 16:49:45 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/30-12-2023_12-34-05_all_train_images/model_final.pth ...


In [4]:
# load segmentsai handlers
from src.annotation_handling.segmentsai_handler import SegmentsAIHandler
from src.pipelines import model_assist_label_pipeline as pipe 
import config
import time
from segments import SegmentsDataset

segmentsai_handler = SegmentsAIHandler()


add a new dataset to segments and predict annotations using the model trained

In [12]:
from src.annotation_handling import annotation_handler

# New dataset to upload
image_name = "IMG_0041"
full_week = 'week5'
zoom_type = '3xr'
release = "v0.1"
nir_username = "Nirmalka"
dorin_username = "dorins"
collaborator = nir_username

# Create a new test dataset for the specified image, week, and zoom type
test_dataset = pipe.create_new_test_dataset(image_name, config.WEEKS_DIR[full_week], config.ZOOM_TYPES_DIR[zoom_type], single_category=False)
# # Get the absolute path to the processed image
abs_images_path = f"{config.get_processed_cannabis_image_path(full_week, zoom_type)}/{image_name}"
raw_images_path = f"{config.get_raw_image_path(full_week, zoom_type)}/{image_name}.JPG"

# Upload the images that are not annotated to the dataset
segmentsai_handler.upload_single_image(test_dataset, raw_images_path) # upload the raw image path
segmentsai_handler.upload_images(test_dataset, abs_images_path)

description = "upload predictions to dataset."
segmentsai_handler.client.add_release(test_dataset, release, description)
print("Waiting 5 seconds that the release is created...")
time.sleep(5)
test_release = segmentsai_handler.client.get_release(test_dataset, "v0.1")
print(test_release)

# Add collaborator to the dataset
segmentsai_handler.client.add_dataset_collaborator(
    dataset_identifier=test_dataset,
    username=collaborator
)

test_dataset_instance = SegmentsDataset(test_release)

for sample in test_dataset_instance:
    # Generate label predictions
    image = np.array(sample["image"])
    # Change RGB (PIL) to BGR (OpenCV)
    cv2_image = image[:, :, ::-1]
    outputs = predictor(cv2_image)
    
    segmentation_bitmap, annotations = annotation_handler.convert_coco_to_segments_format(cv2_image, outputs)
    segmentsai_handler.upload_annotation_for_sample(sample['uuid'], segmentation_bitmap, annotations)
    
# add annotation tracking entry    
config.add_annotation_tracking_entry(image_name, collaborator)

name='cannabis_patches_week5_18_05_2023_3x_regular_IMG_0041' full_name='etaylor/cannabis_patches_week5_18_05_2023_3x_regular_IMG_0041' cloned_from=None description='cannabis patches week=week5_18_05_2023 zoom_type=3x_regular of image=IMG_0041.' category='other' public=False owner=Owner(username='etaylor', created_at='2022-12-28T12:53:18Z', email=None) created_at='2024-04-06T12:27:56.426421Z' enable_ratings=False enable_skip_labeling=True enable_skip_reviewing=False enable_save_button=False enable_label_status_verified=False enable_same_dimensions_track_constraint=False enable_interpolation=True task_type='segmentation-bitmap' label_stats=LabelStats(REVIEWED=None, REVIEWING_IN_PROGRESS=None, LABELED=None, LABELING_IN_PROGRESS=None, REJECTED=None, PRELABELED=None, SKIPPED=None, VERIFIED=None, UNLABELED=None, TOTAL=None) labeling_inactivity_timeout_seconds=300 samples_count=0 collaborators_count=None task_attributes=TaskAttributes(format_version='0.1', categories=[TaskAttributeCategory(na

100%|██████████| 10/10 [00:04<00:00,  2.43it/s]


Initialized dataset with 10 images.


### upload datasets for all missing annotations images

In [5]:
# read pickle file
import pickle

with open("/home/etaylor/code_projects/thesis/metadata/missed_annotation_images.pkl", 'rb') as f:
    missing_annotations_images_list = pickle.load(f)

In [7]:
print(missing_annotations_images_list)

['IMG_0030', 'IMG_0586', 'IMG_2250', 'IMG_1119', 'IMG_0591', 'IMG_2257', 'IMG_1133', 'IMG_0638', 'IMG_1768', 'IMG_1132', 'IMG_1130', 'IMG_0041', 'IMG_1840', 'IMG_1835', 'IMG_2149', 'IMG_0652', 'IMG_2304', 'IMG_0580', 'IMG_0633', 'IMG_0033', 'IMG_0062', 'IMG_1867', 'IMG_1161', 'IMG_2210', 'IMG_2234', 'IMG_1117', 'IMG_1834', 'IMG_0573', 'IMG_1778', 'IMG_0649', 'IMG_0022', 'IMG_1838', 'IMG_1823', 'IMG_1868', 'IMG_1112', 'IMG_1094', 'IMG_1799', 'IMG_2224', 'IMG_2148', 'IMG_1128', 'IMG_1137', 'IMG_2140', 'IMG_0583', 'IMG_1798', 'IMG_0575', 'IMG_9986', 'IMG_0071', 'IMG_2171', 'IMG_9985', 'IMG_1849', 'IMG_2228', 'IMG_0642', 'IMG_1191', 'IMG_1193', 'IMG_1188', 'IMG_0577', 'IMG_1145', 'IMG_2161', 'IMG_1906', 'IMG_2135', 'IMG_1187', 'IMG_1118', 'IMG_2276', 'IMG_1182', 'IMG_0581', 'IMG_2271', 'IMG_2235', 'IMG_1144', 'IMG_0048', 'IMG_2167', 'IMG_2163', 'IMG_1111', 'IMG_0576', 'IMG_9979', 'IMG_2166', 'IMG_1160', 'IMG_2146', 'IMG_0023', 'IMG_1204', 'IMG_2124', 'IMG_2183', 'IMG_1135', 'IMG_2144', 'IM

In [26]:
from src.annotation_handling import annotation_handler
import numpy as np
import time
from segments.exceptions import NotFoundError

# define annotators usernames
nir_username = "Nirmalka"
dorin_username = "dorins"

# Assuming 'missing_annotations_images_list' contains the list of images that need annotations
half_index = len(missing_annotations_images_list) // 2

for index, image_name in enumerate(missing_annotations_images_list):
    
    full_week, full_zoom = config.find_image_details(image_name)
    full_week = full_week.split("_")[0]
    zoom_type = full_zoom.split("_")[0] + "r"
    release = "v0.1"

    # Assign collaborator based on index
    if index < half_index:
        collaborator = nir_username
    else:
        collaborator = dorin_username
        
    print(f"Processing image {image_name}, image index {index} with collaborator {collaborator}")
        
    dataset_name = f"etaylor/cannabis_patches_{config.WEEKS_DIR[full_week]}_{config.ZOOM_TYPES_DIR[zoom_type]}_{image_name}"
    print(f"Dataset name: {dataset_name}")
    # Try to get the dataset, and skip if not found
    try:
        # Attempt to get the dataset
        test_dataset = segmentsai_handler.client.get_dataset(dataset_name)
        print(f"Dataset found: {test_dataset}, skipping upload.")
        continue
    except NotFoundError:
        print(f"Dataset not found for {image_name}, uploading dataset")
        # Create a new test dataset for the specified image, week, and zoom type
        test_dataset = pipe.create_new_test_dataset(image_name, config.WEEKS_DIR[full_week], config.ZOOM_TYPES_DIR[zoom_type], single_category=False)
        # Get the absolute path to the processed image
        abs_images_path = f"{config.get_processed_cannabis_image_path(full_week, zoom_type)}/{image_name}"
        raw_images_path = f"{config.get_raw_image_path(full_week, zoom_type)}/{image_name}.JPG"

        # Upload the images that are not annotated to the dataset
        segmentsai_handler.upload_single_image(test_dataset, raw_images_path) # upload the raw image path
        segmentsai_handler.upload_images(test_dataset, abs_images_path)

        description = "upload predictions to dataset."
        segmentsai_handler.client.add_release(test_dataset, release, description)
        print(f"Waiting 5 seconds that the release {release} is created for {image_name}...")
        time.sleep(5)
        test_release = segmentsai_handler.client.get_release(test_dataset, release)
        print(test_release)

        # Add collaborator to the dataset
        segmentsai_handler.client.add_dataset_collaborator(
            dataset_identifier=test_dataset,
            username=collaborator
        )

        test_dataset_instance = SegmentsDataset(test_release)

        for sample in test_dataset_instance:
            # Generate label predictions
            image = np.array(sample["image"])
            # Change RGB (PIL) to BGR (OpenCV)
            cv2_image = image[:, :, ::-1]
            outputs = predictor(cv2_image)
            
            segmentation_bitmap, annotations = annotation_handler.convert_coco_to_segments_format(cv2_image, outputs)
            segmentsai_handler.upload_annotation_for_sample(sample['uuid'], segmentation_bitmap, annotations)
        


Processing image IMG_0030, image index 0 with collaborator Nirmalka
Dataset name: etaylor/cannabis_patches_week5_18_05_2023_3x_regular_IMG_0030
Dataset found: name='cannabis_patches_week5_18_05_2023_3x_regular_IMG_0030' full_name='etaylor/cannabis_patches_week5_18_05_2023_3x_regular_IMG_0030' cloned_from=None description='cannabis patches week=week5_18_05_2023 zoom_type=3x_regular of image=IMG_0030.' category='other' public=False owner=Owner(username='etaylor', created_at='2022-12-28T12:53:18Z', email=None) created_at='2024-04-16T14:01:38.179727Z' enable_ratings=False enable_skip_labeling=True enable_skip_reviewing=False enable_save_button=False enable_label_status_verified=False enable_same_dimensions_track_constraint=False enable_interpolation=True task_type='segmentation-bitmap' label_stats=LabelStats(REVIEWED=None, REVIEWING_IN_PROGRESS=None, LABELED=None, LABELING_IN_PROGRESS=None, REJECTED=None, PRELABELED=7, SKIPPED=None, VERIFIED=None, UNLABELED=None, TOTAL=7) labeling_inactivi

100%|██████████| 14/14 [00:01<00:00,  9.28it/s]


Initialized dataset with 14 images.
Processing image IMG_2151, image index 87 with collaborator dorins
Dataset name: etaylor/cannabis_patches_week9_15_06_2023_3x_regular_IMG_2151
Dataset not found for IMG_2151, uploading dataset
name='cannabis_patches_week9_15_06_2023_3x_regular_IMG_2151' full_name='etaylor/cannabis_patches_week9_15_06_2023_3x_regular_IMG_2151' cloned_from=None description='cannabis patches week=week9_15_06_2023 zoom_type=3x_regular of image=IMG_2151.' category='other' public=False owner=Owner(username='etaylor', created_at='2022-12-28T12:53:18Z', email=None) created_at='2024-04-16T15:08:36.807418Z' enable_ratings=False enable_skip_labeling=True enable_skip_reviewing=False enable_save_button=False enable_label_status_verified=False enable_same_dimensions_track_constraint=False enable_interpolation=True task_type='segmentation-bitmap' label_stats=LabelStats(REVIEWED=None, REVIEWING_IN_PROGRESS=None, LABELED=None, LABELING_IN_PROGRESS=None, REJECTED=None, PRELABELED=None

100%|██████████| 12/12 [00:02<00:00,  4.89it/s]


Initialized dataset with 12 images.
Processing image IMG_1190, image index 88 with collaborator dorins
Dataset name: etaylor/cannabis_patches_week7_01_06_2023_3x_regular_IMG_1190
Dataset not found for IMG_1190, uploading dataset
name='cannabis_patches_week7_01_06_2023_3x_regular_IMG_1190' full_name='etaylor/cannabis_patches_week7_01_06_2023_3x_regular_IMG_1190' cloned_from=None description='cannabis patches week=week7_01_06_2023 zoom_type=3x_regular of image=IMG_1190.' category='other' public=False owner=Owner(username='etaylor', created_at='2022-12-28T12:53:18Z', email=None) created_at='2024-04-16T15:08:59.451350Z' enable_ratings=False enable_skip_labeling=True enable_skip_reviewing=False enable_save_button=False enable_label_status_verified=False enable_same_dimensions_track_constraint=False enable_interpolation=True task_type='segmentation-bitmap' label_stats=LabelStats(REVIEWED=None, REVIEWING_IN_PROGRESS=None, LABELED=None, LABELING_IN_PROGRESS=None, REJECTED=None, PRELABELED=None

...

Initialized dataset with 11 images.
Processing image IMG_0597, image index 142 with collaborator dorins
Dataset name: etaylor/cannabis_patches_week6_22_05_2023_3x_regular_IMG_0597
Dataset not found for IMG_0597, uploading dataset
name='cannabis_patches_week6_22_05_2023_3x_regular_IMG_0597' full_name='etaylor/cannabis_patches_week6_22_05_2023_3x_regular_IMG_0597' cloned_from=None description='cannabis patches week=week6_22_05_2023 zoom_type=3x_regular of image=IMG_0597.' category='other' public=False owner=Owner(username='etaylor', created_at='2022-12-28T12:53:18Z', email=None) created_at='2024-04-16T15:29:48.282067Z' enable_ratings=False enable_skip_labeling=True enable_skip_reviewing=False enable_save_button=False enable_label_status_verified=False enable_same_dimensions_track_constraint=False enable_interpolation=True task_type='segmentation-bitmap' label_stats=LabelStats(REVIEWED=None, REVIEWING_IN_PROGRESS=None, LABELED=None, LABELING_IN_PROGRESS=None, REJECTED=None, PRELABELED=Non

100%|██████████| 11/11 [00:01<00:00,  7.84it/s]


Initialized dataset with 11 images.
Processing image IMG_1827, image index 143 with collaborator dorins
Dataset name: etaylor/cannabis_patches_week8_07_06_2023_3x_regular_IMG_1827
Dataset not found for IMG_1827, uploading dataset
name='cannabis_patches_week8_07_06_2023_3x_regular_IMG_1827' full_name='etaylor/cannabis_patches_week8_07_06_2023_3x_regular_IMG_1827' cloned_from=None description='cannabis patches week=week8_07_06_2023 zoom_type=3x_regular of image=IMG_1827.' category='other' public=False owner=Owner(username='etaylor', created_at='2022-12-28T12:53:18Z', email=None) created_at='2024-04-16T15:30:09.773958Z' enable_ratings=False enable_skip_labeling=True enable_skip_reviewing=False enable_save_button=False enable_label_status_verified=False enable_same_dimensions_track_constraint=False enable_interpolation=True task_type='segmentation-bitmap' label_stats=LabelStats(REVIEWED=None, REVIEWING_IN_PROGRESS=None, LABELED=None, LABELING_IN_PROGRESS=None, REJECTED=None, PRELABELED=Non

100%|██████████| 13/13 [00:01<00:00,  8.43it/s]


Initialized dataset with 13 images.


In [27]:
# add the annotations to the metadata annotation tracking csv file
for index, image_name in enumerate(missing_annotations_images_list):
   # Assign collaborator based on index
    if index < half_index:
        collaborator = nir_username
    else:
        collaborator = dorin_username
        
    print(f"Adding annotation tracking entry to image {image_name} with annotator {collaborator}...")    
    config.add_annotation_tracking_entry(image_name, collaborator)

Adding annotation tracking entry to image IMG_0030 with annotator Nirmalka...
Adding annotation tracking entry to image IMG_0586 with annotator Nirmalka...
Adding annotation tracking entry to image IMG_2250 with annotator Nirmalka...
Adding annotation tracking entry to image IMG_1119 with annotator Nirmalka...
Adding annotation tracking entry to image IMG_0591 with annotator Nirmalka...
Adding annotation tracking entry to image IMG_2257 with annotator Nirmalka...
Adding annotation tracking entry to image IMG_1133 with annotator Nirmalka...
Adding annotation tracking entry to image IMG_0638 with annotator Nirmalka...
Adding annotation tracking entry to image IMG_1768 with annotator Nirmalka...
Adding annotation tracking entry to image IMG_1132 with annotator Nirmalka...
Adding annotation tracking entry to image IMG_1130 with annotator Nirmalka...
Adding annotation tracking entry to image IMG_0041 with annotator Nirmalka...
Adding annotation tracking entry to image IMG_1840 with annotato